In [26]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import numpy as np

from starter import CNN

import sys
sys.path.insert(1, '../saliency')
from network_saliency import visualize_helper_selftrained 

In [62]:
class VarianceCNN(nn.Module):
    def __init__(self):
        super(VarianceCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(8000, 4000)
        self.fc2 = nn.Linear(4000, 2000)
        self.fc3 = nn.Linear(2000, 784)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2_drop(self.conv2(x)))
        x = x.view(-1, 8000)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [63]:
# Load Original Model

device = torch.device("cpu")
model = CNN().to(device)
lr = .01
momentum = 0.5
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
checkpoint = torch.load("trained_networks/trained_starter")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
model.train()

CNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(2, 2))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(2, 2))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [64]:
# Load MNIST Test Set

test_loader = torch.utils.data.DataLoader( 
    datasets.MNIST('../networks/data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)),
                   ])),
        batch_size=1, shuffle=True)

In [65]:
variance_model = VarianceCNN().to(device)

In [119]:
def my_loss(input_img, output, model):
    input_img = input_img.detach().numpy()
    for i in range(0, 784):
        noise = np.random.normal(0, abs(output[0][i].detach().numpy()))
        input_img[0][0][int(i/28), i%28] = input_img[0][0][int(i/28), i%28]
    new_output = model(torch.from_numpy(input_img).to(device)).argmax(axis=1)
    print(new_output)


In [126]:
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = data.to(device), target.to(device)
    predicted_output = model(data).argmax(axis=1)
    gradients, max_gradients = visualize_helper_selftrained(model, tensor=data, k=predicted_output)
    
    print(target)
    # Have to add a dimension
    gradients = gradients[np.newaxis, ...]
    print(gradients.shape)
    print(data.shape)
    output = variance_model(gradients)
    my_loss(data, output, model)
    break


tensor([1])
torch.Size([1, 1, 28, 28])
torch.Size([1, 1, 28, 28])
tensor([1])
